## Dependencies

In [ ]:
! pip install pandas
! pip install geopandas

In [56]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

from urllib.request import urlretrieve

## Load Census Data

In [9]:
class Census_Data_Provider():
  @classmethod
  def get_tract_data(self) -> gpd.geodataframe:
    if not os.path.exists('content/race_comparison_manhattan.geojson'):
      url = '''https://raw.githubusercontent.com/bicdev/tcc/7c10dd978fcac26e6d6eee38629a25e159697faa/cached%20data/race_comparison_manhattan.geojson'''
      display("downloading cached data from github")
      urlretrieve(url, 'content/race_comparison_manhattan.geojson')
    return gpd.read_file('content/race_comparison_manhattan.geojson')

In [ ]:
def get_tracts_racialized(df_race_census: pd.DataFrame, slice_size: int) -> pd.DataFrame:
  df_racial_delta = df_race_census[[
    'NAME', 
    'geometry',
    'Total:  >  Population of one race:  >  White alone',
    'Total:  >  Population of one race:  >  Black or African American alone',
    'Total:  >  Population of one race:  >  American Indian and Alaska Native alone',
    'Total:  >  Population of one race:  >  Asian alone',
    'Total:  >  Population of one race:  >  Native Hawaiian and Other Pacific Islander alone',
    'Total:  >  Population of one race:  >  Some Other Race alone'
  ]]

  df_racial_delta['Racial Ratio'] = pd.Series()
  df_racial_delta['Racial Label'] = pd.Series()
  
  for i, row in df_racial_delta.iterrows():
    whites = row['Total:  >  Population of one race:  >  White alone']
    non_whites = (
      row['Total:  >  Population of one race:  >  Black or African American alone'] +
      row['Total:  >  Population of one race:  >  American Indian and Alaska Native alone'] +
      row['Total:  >  Population of one race:  >  Asian alone'] +
      row['Total:  >  Population of one race:  >  Native Hawaiian and Other Pacific Islander alone'] +
      row['Total:  >  Population of one race:  >  Some Other Race alone']
    )

    if whites > non_whites:
      label = "Whites"
    elif whites < non_whites:
      label = "Non-Whites"
    else:
      label = "None"

    if whites == 0.0 or non_whites == 0.0:
      ratio = 0
      label = "None"
    else:
      ratio = min(whites, non_whites) / max(whites, non_whites)
    
    df_racial_delta.loc[i,'Racial Ratio'] = ratio
    df_racial_delta.loc[i,'Racial Label'] = label

  df_racial_delta.sort_values(by='Racial Ratio', ascending=False, inplace=True)
  df_racial_delta.reset_index(inplace=True)
  ids_to_drop = df_racial_delta[df_racial_delta['Racial Label'] == "None"].index
  df_racial_delta.drop(ids_to_drop, inplace=True)

  gdf_top_slice_racialized = gpd.GeoDataFrame(
    df_racial_delta,
    geometry=df_racial_delta['geometry'],
    crs=4326)

  return gdf_top_slice_racialized